# &pi; in Julia

original REPO:
https://github.com/simonbyrne/Pi.jl
 
*([Simon Byrne](https://github.com/simonbyrne))*

Like most technical languages, Julia provides a variable constant for &pi;. However Julia's handling is a bit special.

In [1]:
pi

π = 3.1415926535897...

It can also be accessed via the unicode symbol (you can get it at the REPL or in a notebook via the TeX completion `\pi` followed by a tab)

In [2]:
π

π = 3.1415926535897...

You'll notice that it doesn't print like an ordinary floating point number: that's because it isn't one.

In [3]:
typeof(pi)

Irrational{:π}

&pi; and a few other irrational constants are instead stored as special `Irrational` values, rather than being rounded to `Float64`. These act like ordinary numeric values, except that they can are converted automatically to any floating point type without any intermediate rounding:

In [4]:
1 + pi # integers are promoted to Float64 by default

4.141592653589793

In [5]:
Float32(1) + pi # Float32

4.141593f0

This is particularly useful for use with arbitrary-precision `BigFloat`s, as &pi; can be evaluated to full precision (rather than be truncated to `Float64` and converted back).

In [6]:
BigFloat(1) + pi # 256 bits by default

4.141592653589793238462643383279502884197169399375105820974944592307816406286198

If &pi; were stored as a `Float64`, we would instead get

In [7]:
BigFloat(1) + Float64(pi)

4.141592653589793115997963468544185161590576171875

In fact `BigFloat` (which uses the [MPFR](http://www.mpfr.org) library) will compute &pi; on demand to the current precision, which is set via `setprecision`. This provides an easy way to get its digits: 

In [8]:
# to 1024 bits
setprecision(BigFloat, 1024) do 
    BigFloat(pi)
end

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117067982148086513282306647093844609550582231725359408128481117450284102701938521105559644622948954930381964428810975665933446128475648233786783165271201909145648566923460348610454326648213393607260249141273724586997

The last few digits may be incorrect due to the conversion from the internal binary format of `BigFloat` to the decimal representation used for printing.

Another neat property of `Irrational`s is that inequalities are correct:

In [9]:
Float64(pi) < pi < nextfloat(Float64(pi))

true

## &pi; via inline assembly instructions
*([Simon Byrne](https://github.com/simonbyrne))*

Julia provides a very low-level `llvmcall` interface, which allows the user to directly write [LLVM intermediate representation](http://llvm.org/docs/LangRef.html), including the use of inline assembly. The following snippet calls the `fldpi` instruction ("**f**loating point **l**oa**d** **pi**") which loads the constant &pi; onto the floating point register stack (this works only on x86 and x86_64 architectures)

In [10]:
function asm_pi()
    Base.llvmcall(
    """ %pi = call double asm "fldpi", "={st}"()
        ret double %pi""", 
    Float64, Tuple{})
end

asm_pi (generic function with 1 method)

In [11]:
asm_pi()

3.141592653589793

We can look at the actual resulting code that is generated:

In [12]:
@code_native asm_pi()

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 13, 0
	.globl	_julia_asm_pi_1235              ## -- Begin function julia_asm_pi_1235
	.p2align	4, 0x90
_julia_asm_pi_1235:                     ## @julia_asm_pi_1235
; ┌ @ In[10]:1 within `asm_pi`
	.cfi_startproc
## %bb.0:                               ## %top
; │ @ In[10]:2 within `asm_pi`
	## InlineAsm Start
	fldpi
	## InlineAsm End
	fstpl	-8(%rsp)
	movsd	-8(%rsp), %xmm0                 ## xmm0 = mem[0],zero
	retq
	.cfi_endproc
; └
                                        ## -- End function
.subsections_via_symbols


If you're wondering what the rest of these instructions are doing:

1. the `pushq` and `movq` adds to the [call stack frame](https://en.wikipedia.org/wiki/Call_stack).
2. `fldpi` pushes &pi; to the x87 floating point register stack
  - x87 is the older legacy floating point instruction set dating back to the original [Intel 8087 coprocessor](https://en.wikipedia.org/wiki/Intel_8087).
3. `fstpl` and `movsd` moves the value to the SSE floating point register `xmm0`
  - Julia, like most modern software, uses the newer SSE instruction set for its floating point operations. This also allows us to take advantage of things like [SIMD operations](https://en.wikipedia.org/wiki/SIMD).
4. `popq` and `retq` pops the call stack frame.

## &pi; using a Taylor series expansions
*([Luis Benet](https://github.com/lbenet), Instituto de Ciencias Físicas, Universidad Nacional Autónoma de México (UNAM))*

This will demonstrate how to evaluate &pi; using various Taylor series expansions via the [TaylorSeries.jl](https://github.com/JuliaDiff/TaylorSeries.jl) package.

In [13]:
using TaylorSeries

LoadError: ArgumentError: Package TaylorSeries not found in current path.
- Run `import Pkg; Pkg.add("TaylorSeries")` to install the TaylorSeries package.

### Madhava's formula

One of the standard trigonmetric identities is
$$ \tan\left( \frac{\pi}{6} \right) = \frac{1}{\sqrt{3}}. $$

Therefore, by taking the Taylor expansion of $6 \arctan(x)$ around 0 we may obtain the value of $\pi$, by evaluating it at $1/\sqrt{3}$, a value which is within the radius of convergence.

We obtain the Taylor series of order 37th, using `BigFloat`s:

In [14]:
series1 = 6atan( Taylor1(BigFloat, 37) )
convert(Taylor1{Rational{BigInt}},series1)

LoadError: UndefVarError: `Taylor1` not defined

Note that the series above has only odd powers, so we will be using in this case 18 coefficients.

Evaluating that expression in $1/\sqrt{3}$ we get

In [15]:
pi_approx1 = evaluate(series1, 1/sqrt(big(3)))

LoadError: UndefVarError: `series1` not defined

Then, the 37th order Taylor expansion yields a value which differs from $\pi$ in:

In [16]:
abs(pi - pi_approx1)

LoadError: UndefVarError: `pi_approx1` not defined

To obtain more accurate results, we may simply increase the order of the expansion:

In [17]:
series2 = 6atan( Taylor1(BigFloat,99) ) # 49 coefficients of the series
pi_approx2 = evaluate(series2, 1/sqrt(BigInt(3)))

LoadError: UndefVarError: `Taylor1` not defined

In [18]:
abs(pi - pi_approx2)

LoadError: UndefVarError: `pi_approx2` not defined

This formulation is one of the [*Madhava* or *Gregory–Leibniz series*](https://en.wikipedia.org/wiki/Madhava_series#Another_formula_for_the_circumference_of_a_circle):

\begin{equation}
\pi = 6 \sum_{n=0}^{\infty} (-1)^n \frac{(1/\sqrt{3})^{2n+1}}{2n+1}.
\end{equation}

### Machin's approach

Following the same idea, [John Machin](https://en.wikipedia.org/wiki/John_Machin#Formula) derived an algorithm which converges much faster, using the identity

\begin{equation}
\frac{\pi}{4} = 4 \arctan\left(\frac{1}{5}\right) - \arctan\left(\frac{1}{239}\right).
\end{equation}

Following what we did above, using again a 37th Taylor expansion:

In [19]:
ser = atan( Taylor1(BigFloat, 37) )
pi_approx3 = 4*( 4*evaluate(ser, 1/big(5)) - evaluate(ser, 1/big(239)) )

LoadError: UndefVarError: `Taylor1` not defined

In [20]:
abs(pi - pi_approx3)

LoadError: UndefVarError: `pi_approx3` not defined

## Finding guaranteed bounds on &pi;
*([David P. Sanders](http://sistemas.fciencias.unam.mx/~dsanders/),  Department of Physics, Faculty of Sciences, National University of Mexico (UNAM)*) 

### Using standard floating-point arithmetic 

We will calculate *guaranteed* (i.e., *validated*, or mathematically rigorous) bounds on $\pi$ using just floating-point arithmetic. This requires "directed rounding", i.e. the ability to control in which direction floating-point operations are rounded.

This is based on the book [*Validated Numerics*](http://press.princeton.edu/titles/9488.html) (Princeton, 2011) by [Warwick Tucker](http://www2.math.uu.se/~warwick/CAPA/warwick/warwick.html).

Consider the infinite series

$$ S := \sum_{n=1}^\infty \frac{1}{n^2},$$

whose exact value is [known](http://en.wikipedia.org/wiki/Basel_problem) to be $S = \frac{\pi^2}{6}$.
Thus, if finding guaranteed bounds on $S$ will give guaranteed bounds on $\pi$. 

The idea is to split $S$ up into two parts, $S = S_N + T_N$, where
$ S_N := \sum_{n=1}^N \frac{1}{n^2}$ contains the first $N$ terms, 
and $T_N := S - S_N = \sum_{n=N+1}^\infty \frac{1}{n^2}$ contains the rest (an infinite number of terms).

We will evalute $S_N$ numerically, and use the following analytical bound for $T_N$:

$$\frac{1}{N+1} \le T_N \le \frac{1}{N}$$.

This is obtained by approximating the sum in $T_N$ using integrals from below and above:

$$\int_{x=N+1}^\infty \frac{1}{x^2} dx \le T_N \le \int_{x=N}^\infty \frac{1}{x^2} dx.$$

$S_N$ may be calculated easily by summing either forwards or backwards:

In [21]:
function forward_sum(N, T=Float64)
    total = zero(T)
    for i in 1:N
        total += one(T) / (i^2)
    end
    total
end

function reverse_sum(N, T=Float64)
    total = zero(T)
    for i in N:-1:1
        total += one(T) / (i^2)
    end
    total
end

reverse_sum (generic function with 2 methods)

To find *rigorous* bounds for $S_N$, we use "directed rounding", that is, we round downwards for the lower bound and  upwards for the upper bound:

In [22]:
N = 10^6

lowerbound_S_N = 
    setrounding(Float64, RoundDown) do
        forward_sum(N)
    end

upperbound_S_N = 
    setrounding(Float64, RoundUp) do
        forward_sum(N)
    end

(lowerbound_S_N, upperbound_S_N)

LoadError: MethodError: no method matching setrounding(::Type{Float64}, ::RoundingMode{:Down})

[0mClosest candidates are:
[0m  setrounding([91m::Type{BigFloat}[39m, ::RoundingMode)
[0m[90m   @[39m [90mBase[39m [90m[4mmpfr.jl:134[24m[39m


We incorporate the respective bound on $T_N$ to obtain the bounds on $S$, and hence on $\pi$:

In [23]:
N = 10^6

lower_π =
    setrounding(Float64, RoundDown) do
        lower_bound = forward_sum(N) + 1/(N+1)
        sqrt(6 * lower_bound)
    end

upper_π = 
    setrounding(Float64, RoundUp) do
        upper_bound = forward_sum(N) + 1/N
        sqrt(6 * upper_bound)
    end

(lower_π, upper_π, lowerbound_S_N)

LoadError: MethodError: no method matching setrounding(::Type{Float64}, ::RoundingMode{:Down})

[0mClosest candidates are:
[0m  setrounding([91m::Type{BigFloat}[39m, ::RoundingMode)
[0m[90m   @[39m [90mBase[39m [90m[4mmpfr.jl:134[24m[39m


In [24]:
upper_π - lower_π

LoadError: UndefVarError: `upper_π` not defined

We may check that the true value of $\pi$ is indeed contained in the interval:

In [25]:
lower_π < pi < upper_π

LoadError: UndefVarError: `lower_π` not defined

Summing in the opposite direction turns out to give a more accurate answer:

In [26]:
N = 10^6

lower_π =
    setrounding(Float64, RoundDown) do
        lower_bound = reverse_sum(N) + 1/(N+1)
        sqrt(6 * lower_bound)
    end

upper_π = 
    setrounding(Float64, RoundUp) do
        upper_bound = reverse_sum(N) + 1/N
        sqrt(6 * upper_bound)
    end

(lower_π, upper_π)

LoadError: MethodError: no method matching setrounding(::Type{Float64}, ::RoundingMode{:Down})

[0mClosest candidates are:
[0m  setrounding([91m::Type{BigFloat}[39m, ::RoundingMode)
[0m[90m   @[39m [90mBase[39m [90m[4mmpfr.jl:134[24m[39m


In [27]:
upper_π - lower_π

LoadError: UndefVarError: `upper_π` not defined

In [28]:
lower_π < pi < upper_π

LoadError: UndefVarError: `lower_π` not defined

In principle, we could attain arbitrarily good precision with higher-precision `BigFloat`s, but the result is hampered by the slow convergence of the series.

## Summing a series using interval arithmetic

We repeat the calculation using *interval arithmetic*, provided by the [ValidatedNumerics.jl](https://github.com/dpsanders/ValidatedNumerics.jl) package.

In [29]:
using ValidatedNumerics

LoadError: ArgumentError: Package ValidatedNumerics not found in current path.
- Run `import Pkg; Pkg.add("ValidatedNumerics")` to install the ValidatedNumerics package.

In [30]:
setdisplay(:standard)  # abbreviated display of intervals

LoadError: UndefVarError: `setdisplay` not defined

In [31]:
N = 10000
S = forward_sum(N, Interval)
S += 1/(N+1) .. 1/N  # interval bound on the remainder of the series
π_interval = √(6S)

LoadError: UndefVarError: `Interval` not defined

Here we used an abbreviated display for the interval. Let's see the whole thing:

In [32]:
setdisplay(:full)
π_interval

LoadError: UndefVarError: `setdisplay` not defined

It's diameter (width) is

In [33]:
diam(π_interval)

LoadError: UndefVarError: `diam` not defined

Thus, the result is correct to approximately 8 decimals.

In this calculation, we used the fact that arithmetic operations of intervals with numbers automatically promote the numbers to an interval:

In [34]:
setdisplay(:full)  # full interval display
Interval(0) + 1/3^2

LoadError: UndefVarError: `setdisplay` not defined

This is an interval containing the true real number $1/9$ (written `1//9` in Julia):

In [35]:
1//9 ∈ convert(Interval{Float64}, 1/3^2)

LoadError: UndefVarError: `Interval` not defined

Finally, we can check that the true value of $\pi$ is indeed inside our interval:

In [36]:
pi ∈ π_interval

LoadError: UndefVarError: `π_interval` not defined

## Calculating an area

Although the calculation above is simple, the derivation of the series itself is not. In this section, we will use a more natural way to calculate $\pi$, namely that the area of a circle of radius $r$ is $A(r) = \pi r^2$. We will calculate the area of one quadrant of a circle of radius $r=2$, which is equal to $\pi$:

In [ ]:
using Plots; gr();

[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]


In [ ]:
f(x) = √(4 - x^2)

In [ ]:
plot(f, 0, 2, aspect_ratio=:equal, fill=(0, :orange), alpha=0.2, label="")

The circle of radius $r=2$ is given by $x^2 + y^2 = 2^2 = 4$, so 

$$\pi = \frac{1}{4} A(2) = \int_{x=0}^2 y(x) \, dx = \int_{x=0}^2 \sqrt{4 - x^2}.$$

In calculus, we learn that we can approximate integrals using **Riemann sums**. Interval arithmetic allows us to make these Riemann sums **rigorous** in a very simple way, as follows.

We split up the $x$ axis into intervals, for example of equal width:

In [ ]:
function make_intervals(N=10)
    xs = linspace(0, 2, N+1)
    return [xs[i]..xs[i+1] for i in 1:length(xs)-1]
end

intervals = make_intervals()

Given one of those intervals, we evaluate the function of interest:

In [ ]:
II = intervals[1]

In [ ]:
f(II)

The result is an interval that is **guaranteed to contain** the true range of the function $f$ over that interval. So the lower and upper bounds of the intervals may be used as lower and upper bounds of the height of the box in a Riemann integral:

In [ ]:
intervals = make_intervals(30)

p = plot(aspect_ratio=:equal)
for X in intervals
    Y = f(X)
    
    plot!(IntervalBox(X, Interval(0, Y.lo)), c=:blue, label="", alpha=0.1)
    plot!(IntervalBox(X, Interval(Y.lo, Y.hi)), c=:red, label="", alpha=0.1)
end

plot!(f, 0, 2)

p

Now we just sum up the areas:

In [ ]:
N = 20
intervals = make_intervals(N)

width = 2/N
width * sum(√(4 - X^2) for X in intervals)

As we increase the number of sub-intervals, the approximation gets better and better:

In [ ]:
setdisplay(:standard, sigfigs=5)

println("N \t area interval \t \t diameter")
for N in 50:50:1000
    intervals = make_intervals(N)
    area = (2/N) * sum(√(4 - X^2) for X in intervals)
            
    println("$N \t $area \t $(diam(area))")
end